In [32]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D,concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop

import tensorflow as tf

import keras.backend as K
import pandas as pd
import matplotlib.pyplot as plt

import sys

import numpy as np

cnt=0
T = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\3\split3new_s3b.csv", header=None)
C = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\3\split3news3_benign.csv",header=None)
f3 = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\benign\benign3_b.csv", header=None)


T1 = T.append (f3)
T1.fillna(0,inplace=True)
T1.replace([np.inf,-np.inf],1,inplace=True)

testT = np.array(T1)
y_test = np.array(C)
X_test = np.reshape(testT, (testT.shape[0],32,24))

sample1 = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\3\sample2.csv", header=None)

sampley = np.array(sample1)

class generatorModel (Model):
    def __init__(self):
        #
        super(generatorModel, self).__init__()
       
        self.img_rows = 32
        self.img_cols = 24
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        
        self.la1 = Dense(128 * 8 * 6, activation="relu", input_dim=self.latent_dim)
        self.la2 = Reshape((8, 6, 128))
        self.la3 = UpSampling2D()
        self.la4 = Conv2D(104, kernel_size=4, padding="same")
        self.la5 = BatchNormalization(momentum=0.8)
        self.la6 = Activation("relu")
        self.la7 = UpSampling2D()
        self.la9 = Conv2D(64, kernel_size=4, padding="same")
        self.la10 = BatchNormalization(momentum=0.8)
        self.la11 = Activation("relu")
        self.la12 = Conv2D(self.channels, kernel_size=4, padding="same")
        self.la13 = Activation("tanh")
        
        self.cla1 = Dense(128 * 8 * 6, activation="relu", input_dim=10)
        self.cla2 = Reshape((8, 6, 128))
        self.cla3 = UpSampling2D()
        self.cla4 = Conv2D(24, kernel_size=4, padding="same")
        #self.concat1 = concatenate()

    def call(self, x1,x2):
        #
        #x = self.flatten1(x)
        
        x1 = self.la1(x1)
        x1 = self.la2(x1)
        x1 = self.la3(x1)
        x1 = self.la4(x1)
        
        x2 = self.cla1(x2)
        x2 = self.cla2(x2)
        x2 = self.cla3(x2)
        x2 = self.cla4(x2)
        
        x1 = concatenate([x1,x2])
        
        
        x1 = self.la5(x1)
        x1 = self.la6(x1)
        x1 = self.la7(x1)
       # x1 = self.la8(x1)
        x1 = self.la9(x1)
        x1 = self.la10(x1)
        x1 = self.la11(x1)
        x1 = self.la12(x1)
        x1 = self.la13(x1)
       
        return x1

    
    

class WGAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 24
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z1 =  Input(shape=(self.latent_dim,))
        z2 =  Input(shape=(10,))
        img = self.generator([z1,z2]) 
        
        z2ex = tf.expand_dims(z2,axis=1)
        z2ex = tf.expand_dims(z2ex,axis=-1)
        z2ex = tf.tile(z2ex,[1,32,1,1])
        imgex = tf.concat ([img,z2ex],axis = 2)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        valid = self.critic(imgex)

        # The combined model  (stacked generator and critic)
        self.combined = Model([z1,z2], valid)
        self.combined.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = generatorModel()

        noise = Input(shape=(self.latent_dim,))
        z2 =  Input(shape=(10,))
        
        #model.build(((100,),(10,)))
       # model.summary()
        
        img = model(noise,z2)
        
        return Model([noise,z2], img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=(32, 34, self.channels), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=(32, 34, self.channels))
        
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        #(X_train, _), (_, _) = mnist.load_data()
        X_train = X_test
        global cnt
        # Rescale -1 to 1
       # X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        for epoch in range(epochs):

            for _ in range(self.n_critic):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                input1 = y_test[idx]
                
                z2ex = tf.expand_dims(input1,axis=1)
                z2ex = tf.expand_dims(z2ex,axis=-1)
                z2ex = tf.tile(z2ex,[1,32,1,1])
                imgsex = tf.concat ([imgs,z2ex],axis = 2)
                
                

                # Generate a batch of new images
                gen_imgs = self.generator.predict([noise,input1])
                
                gen_imgsex = tf.concat ([gen_imgs,z2ex],axis = 2)

                # Train the critic
                d_loss_real = self.critic.train_on_batch(imgsex, valid)
                d_loss_fake = self.critic.train_on_batch(gen_imgsex, fake)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                # Clip critic weights
                for l in self.critic.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)


            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch([noise, input1], valid)

            # Plot the progress
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                cnt=cnt+1

    def sample_images(self, epoch):
        r, c = 50, 50
        idx = np.random.randint(0, sampley.shape[0], 2500)
        noise = np.random.normal(0, 1, (14461, self.latent_dim))
        input2 = sampley
        gen_imgs = self.generator.predict([noise,input2])
        
        gen_imgs = np.reshape(gen_imgs, (gen_imgs.shape[0],-1))
        print ( gen_imgs.shape )
        df1 = pd.DataFrame(gen_imgs)
        df1.to_csv (r'C:\Users\maijieai\pratice\p1\tensorf\gan_try\samplest'+'\\'+str(cnt)+'.csv')
        # Rescale images 0 - 1
        #gen_imgs = 0.5 * gen_imgs + 0.5
        '''
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()
        '''
       


if __name__ == '__main__':
    wgan = WGAN()
    wgan.train(epochs=8001, batch_size=50, sample_interval=2000)

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_212 (Conv2D)         (None, 16, 17, 16)        160       
                                                                 
 leaky_re_lu_108 (LeakyReLU)  (None, 16, 17, 16)       0         
                                                                 
 dropout_108 (Dropout)       (None, 16, 17, 16)        0         
                                                                 
 conv2d_213 (Conv2D)         (None, 8, 9, 32)          4640      
                                                                 
 zero_padding2d_27 (ZeroPadd  (None, 9, 10, 32)        0         
 ing2D)                                                          
                                                                 
 batch_normalization_133 (Ba  (None, 9, 10, 32)        128       
 tchNormalization)                                   

125 [D loss: 1.002012] [G loss: 1.000232]
126 [D loss: 1.002054] [G loss: 1.000074]
127 [D loss: 1.001895] [G loss: 1.000103]
128 [D loss: 1.002025] [G loss: 1.000057]
129 [D loss: 1.002099] [G loss: 1.000113]
130 [D loss: 1.002015] [G loss: 1.000211]
131 [D loss: 1.001900] [G loss: 1.000187]
132 [D loss: 1.002147] [G loss: 1.000050]
133 [D loss: 1.001989] [G loss: 1.000131]
134 [D loss: 1.002161] [G loss: 1.000143]
135 [D loss: 1.002154] [G loss: 1.000022]
136 [D loss: 1.002042] [G loss: 1.000124]
137 [D loss: 1.002069] [G loss: 1.000180]
138 [D loss: 1.002095] [G loss: 1.000198]
139 [D loss: 1.002072] [G loss: 1.000145]
140 [D loss: 1.002171] [G loss: 1.000060]
141 [D loss: 1.002268] [G loss: 1.000090]
142 [D loss: 1.002081] [G loss: 1.000189]
143 [D loss: 1.002219] [G loss: 1.000109]
144 [D loss: 1.002178] [G loss: 1.000191]
145 [D loss: 1.002272] [G loss: 1.000227]
146 [D loss: 1.002253] [G loss: 1.000136]
147 [D loss: 1.002292] [G loss: 1.000144]
148 [D loss: 1.002376] [G loss: 1.

321 [D loss: 1.003060] [G loss: 1.000143]
322 [D loss: 1.003066] [G loss: 1.000068]
323 [D loss: 1.002916] [G loss: 0.999912]
324 [D loss: 1.003137] [G loss: 1.000306]
325 [D loss: 1.002807] [G loss: 1.000454]
326 [D loss: 1.002746] [G loss: 1.000715]
327 [D loss: 1.002730] [G loss: 1.000570]
328 [D loss: 1.002808] [G loss: 1.000495]
329 [D loss: 1.002797] [G loss: 1.000630]
330 [D loss: 1.002974] [G loss: 1.000573]
331 [D loss: 1.002751] [G loss: 1.000645]
332 [D loss: 1.002924] [G loss: 1.000446]
333 [D loss: 1.002937] [G loss: 1.000414]
334 [D loss: 1.002739] [G loss: 1.000452]
335 [D loss: 1.002960] [G loss: 1.000569]
336 [D loss: 1.002902] [G loss: 1.000446]
337 [D loss: 1.002963] [G loss: 1.000426]
338 [D loss: 1.003019] [G loss: 1.000600]
339 [D loss: 1.002892] [G loss: 1.000371]
340 [D loss: 1.003070] [G loss: 1.000390]
341 [D loss: 1.002867] [G loss: 1.000519]
342 [D loss: 1.002711] [G loss: 1.000548]
343 [D loss: 1.002908] [G loss: 1.000459]
344 [D loss: 1.002691] [G loss: 1.

517 [D loss: 1.002894] [G loss: 1.000319]
518 [D loss: 1.002827] [G loss: 1.000264]
519 [D loss: 1.002990] [G loss: 1.000379]
520 [D loss: 1.002796] [G loss: 1.000478]
521 [D loss: 1.003104] [G loss: 1.000333]
522 [D loss: 1.003144] [G loss: 1.000387]
523 [D loss: 1.003061] [G loss: 1.000350]
524 [D loss: 1.002929] [G loss: 1.000345]
525 [D loss: 1.002996] [G loss: 1.000543]
526 [D loss: 1.002885] [G loss: 1.000198]
527 [D loss: 1.002925] [G loss: 1.000479]
528 [D loss: 1.002823] [G loss: 1.000459]
529 [D loss: 1.003009] [G loss: 1.000270]
530 [D loss: 1.003228] [G loss: 1.000520]
531 [D loss: 1.002855] [G loss: 1.000307]
532 [D loss: 1.003029] [G loss: 1.000675]
533 [D loss: 1.003157] [G loss: 1.000316]
534 [D loss: 1.003070] [G loss: 1.000367]
535 [D loss: 1.002815] [G loss: 1.000323]
536 [D loss: 1.002965] [G loss: 1.000410]
537 [D loss: 1.002957] [G loss: 1.000577]
538 [D loss: 1.003151] [G loss: 1.000470]
539 [D loss: 1.003173] [G loss: 1.000398]
540 [D loss: 1.003002] [G loss: 1.

713 [D loss: 1.002829] [G loss: 1.000335]
714 [D loss: 1.002736] [G loss: 1.000198]
715 [D loss: 1.003002] [G loss: 1.000450]
716 [D loss: 1.002957] [G loss: 1.000247]
717 [D loss: 1.002943] [G loss: 1.000463]
718 [D loss: 1.002897] [G loss: 1.000435]
719 [D loss: 1.002969] [G loss: 1.000347]
720 [D loss: 1.002986] [G loss: 1.000469]
721 [D loss: 1.003031] [G loss: 1.000329]
722 [D loss: 1.002911] [G loss: 1.000399]
723 [D loss: 1.002985] [G loss: 1.000317]
724 [D loss: 1.003043] [G loss: 1.000559]
725 [D loss: 1.002905] [G loss: 1.000550]
726 [D loss: 1.002871] [G loss: 1.000508]
727 [D loss: 1.002937] [G loss: 1.000258]
728 [D loss: 1.002979] [G loss: 1.000212]
729 [D loss: 1.002790] [G loss: 1.000621]
730 [D loss: 1.002846] [G loss: 1.000274]
731 [D loss: 1.002852] [G loss: 1.000295]
732 [D loss: 1.003065] [G loss: 1.000499]
733 [D loss: 1.002807] [G loss: 1.000672]
734 [D loss: 1.002836] [G loss: 1.000288]
735 [D loss: 1.002776] [G loss: 1.000412]
736 [D loss: 1.002956] [G loss: 1.

...

7214 [D loss: 1.003146] [G loss: 1.001008]
7215 [D loss: 1.003122] [G loss: 1.000710]
7216 [D loss: 1.003199] [G loss: 1.000554]
7217 [D loss: 1.003349] [G loss: 1.000906]
7218 [D loss: 1.003072] [G loss: 1.000701]
7219 [D loss: 1.003260] [G loss: 1.000667]
7220 [D loss: 1.003037] [G loss: 1.001148]
7221 [D loss: 1.003119] [G loss: 1.001094]
7222 [D loss: 1.003108] [G loss: 1.000885]
7223 [D loss: 1.002966] [G loss: 1.001002]
7224 [D loss: 1.003180] [G loss: 1.001056]
7225 [D loss: 1.003084] [G loss: 1.000773]
7226 [D loss: 1.003114] [G loss: 1.000819]
7227 [D loss: 1.003415] [G loss: 1.001340]
7228 [D loss: 1.003142] [G loss: 1.000802]
7229 [D loss: 1.003067] [G loss: 1.001074]
7230 [D loss: 1.003209] [G loss: 1.000643]
7231 [D loss: 1.003217] [G loss: 1.000752]
7232 [D loss: 1.003044] [G loss: 1.000760]
7233 [D loss: 1.003054] [G loss: 1.001064]
7234 [D loss: 1.002994] [G loss: 1.000770]
7235 [D loss: 1.003063] [G loss: 1.000868]
7236 [D loss: 1.003214] [G loss: 1.000782]
7237 [D los

7405 [D loss: 1.003018] [G loss: 1.000786]
7406 [D loss: 1.003313] [G loss: 1.000673]
7407 [D loss: 1.003079] [G loss: 1.001026]
7408 [D loss: 1.002886] [G loss: 1.000792]
7409 [D loss: 1.003200] [G loss: 1.000642]
7410 [D loss: 1.003262] [G loss: 1.000785]
7411 [D loss: 1.003158] [G loss: 1.000847]
7412 [D loss: 1.003129] [G loss: 1.000429]
7413 [D loss: 1.003274] [G loss: 1.000656]
7414 [D loss: 1.002990] [G loss: 1.000769]
7415 [D loss: 1.003266] [G loss: 1.000701]
7416 [D loss: 1.003058] [G loss: 1.000740]
7417 [D loss: 1.003208] [G loss: 1.000737]
7418 [D loss: 1.003050] [G loss: 1.000911]
7419 [D loss: 1.003318] [G loss: 1.000394]
7420 [D loss: 1.003085] [G loss: 1.000879]
7421 [D loss: 1.003147] [G loss: 1.000631]
7422 [D loss: 1.003173] [G loss: 1.000802]
7423 [D loss: 1.003200] [G loss: 1.000603]
7424 [D loss: 1.003295] [G loss: 1.000853]
7425 [D loss: 1.002946] [G loss: 1.001216]
7426 [D loss: 1.003280] [G loss: 1.000979]
7427 [D loss: 1.003102] [G loss: 1.000542]
7428 [D los

7596 [D loss: 1.003173] [G loss: 1.000774]
7597 [D loss: 1.003097] [G loss: 1.000969]
7598 [D loss: 1.003180] [G loss: 1.000760]
7599 [D loss: 1.003269] [G loss: 1.000658]
7600 [D loss: 1.003146] [G loss: 1.000583]
7601 [D loss: 1.003134] [G loss: 1.000956]
7602 [D loss: 1.003282] [G loss: 1.000779]
7603 [D loss: 1.002993] [G loss: 1.001168]
7604 [D loss: 1.003039] [G loss: 1.001016]
7605 [D loss: 1.003252] [G loss: 1.000993]
7606 [D loss: 1.003266] [G loss: 1.000798]
7607 [D loss: 1.003100] [G loss: 1.000909]
7608 [D loss: 1.003329] [G loss: 1.000905]
7609 [D loss: 1.003128] [G loss: 1.000946]
7610 [D loss: 1.003035] [G loss: 1.000895]
7611 [D loss: 1.003311] [G loss: 1.001015]
7612 [D loss: 1.003281] [G loss: 1.001111]
7613 [D loss: 1.003415] [G loss: 1.001056]
7614 [D loss: 1.003195] [G loss: 1.000941]
7615 [D loss: 1.003446] [G loss: 1.001236]
7616 [D loss: 1.003240] [G loss: 1.000863]
7617 [D loss: 1.003273] [G loss: 1.000789]
7618 [D loss: 1.003261] [G loss: 1.000837]
7619 [D los

7787 [D loss: 1.003125] [G loss: 1.000816]
7788 [D loss: 1.003149] [G loss: 1.000939]
7789 [D loss: 1.003052] [G loss: 1.000675]
7790 [D loss: 1.003382] [G loss: 1.001018]
7791 [D loss: 1.003068] [G loss: 1.000897]
7792 [D loss: 1.003037] [G loss: 1.000891]
7793 [D loss: 1.003290] [G loss: 1.001150]
7794 [D loss: 1.003089] [G loss: 1.001108]
7795 [D loss: 1.003047] [G loss: 1.000745]
7796 [D loss: 1.003269] [G loss: 1.000939]
7797 [D loss: 1.003278] [G loss: 1.001064]
7798 [D loss: 1.003168] [G loss: 1.001341]
7799 [D loss: 1.003201] [G loss: 1.000954]
7800 [D loss: 1.002995] [G loss: 1.000917]
7801 [D loss: 1.003149] [G loss: 1.000811]
7802 [D loss: 1.003174] [G loss: 1.000797]
7803 [D loss: 1.003308] [G loss: 1.000835]
7804 [D loss: 1.003282] [G loss: 1.000732]
7805 [D loss: 1.003285] [G loss: 1.000862]
7806 [D loss: 1.003182] [G loss: 1.000964]
7807 [D loss: 1.003355] [G loss: 1.000992]
7808 [D loss: 1.003174] [G loss: 1.000891]
7809 [D loss: 1.003112] [G loss: 1.000715]
7810 [D los

7978 [D loss: 1.003233] [G loss: 1.000779]
7979 [D loss: 1.002929] [G loss: 1.000748]
7980 [D loss: 1.003073] [G loss: 1.001030]
7981 [D loss: 1.003282] [G loss: 1.001108]
7982 [D loss: 1.003351] [G loss: 1.000832]
7983 [D loss: 1.003173] [G loss: 1.000849]
7984 [D loss: 1.003227] [G loss: 1.000986]
7985 [D loss: 1.002953] [G loss: 1.000454]
7986 [D loss: 1.003208] [G loss: 1.000889]
7987 [D loss: 1.003124] [G loss: 1.000682]
7988 [D loss: 1.003155] [G loss: 1.000925]
7989 [D loss: 1.003080] [G loss: 1.000926]
7990 [D loss: 1.003174] [G loss: 1.001252]
7991 [D loss: 1.003130] [G loss: 1.001029]
7992 [D loss: 1.003131] [G loss: 1.000693]
7993 [D loss: 1.003032] [G loss: 1.001133]
7994 [D loss: 1.003123] [G loss: 1.000770]
7995 [D loss: 1.003477] [G loss: 1.000894]
7996 [D loss: 1.003022] [G loss: 1.001103]
7997 [D loss: 1.003303] [G loss: 1.001149]
7998 [D loss: 1.002955] [G loss: 1.001069]
7999 [D loss: 1.003057] [G loss: 1.000879]
8000 [D loss: 1.003057] [G loss: 1.001299]
(14461, 768